In [ ]:
!pip install --upgrade pip
!pip install numpy==1.26.4
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_groq==0.1.5 arxiv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_groq-0.1.5-py3-none-any.whl.metadata (2.8 kB)
  Using cached arxiv-2.3.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at 

In [ ]:
import os

# Groq API Key (REQUIRED)
os.environ["GROQ_API_KEY"] = " "

# Serper API Key (REQUIRED since you want to use Serper)
os.environ["SERPER_API_KEY"] = " "


In [ ]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from langchain_groq import ChatGroq
import arxiv


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:937: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warnings.warn(


In [ ]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()


In [ ]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.2,
    max_tokens=600
)


In [ ]:
literature_miner = Agent(
    role="Literature Miner",
    goal=(
        "Discover recent and influential research papers using arXiv, "
        "Google search results, and scholarly abstracts."
    ),
    backstory="Expert in academic search and systematic literature review.",
    tools=[search_tool, scrape_tool],
    llm=llm,
    verbose=True
)


In [ ]:
methodology_analyzer = Agent(
    role="Methodology Analyzer",
    goal=(
        "Analyze and compare methodologies across research papers, "
        "focusing on models, datasets, and evaluation strategies."
    ),
    backstory="Experienced research scientist skilled in method comparison.",
    llm=llm,
    verbose=True
)


In [ ]:
trend_synthesizer = Agent(
    role="Trend Synthesizer",
    goal=(
        "Identify emerging trends, dominant techniques, and evolving "
        "research directions from the literature."
    ),
    backstory="Specialist in scientific trend analysis.",
    llm=llm,
    verbose=True
)


In [ ]:
research_gap_agent = Agent(
    role="Research Gap Agent",
    goal=(
        "Identify under-explored research gaps and propose concrete "
        "experiments or benchmarks."
    ),
    backstory="Focused on uncovering novel research opportunities.",
    llm=llm,
    verbose=True
)


In [ ]:
task_1 = Task(
    description=(
        "Search for recent (last 3–5 years) research papers on the topic: {topic}. "
        "Summarize key contributions and datasets used."
    ),
    expected_output=(
        "A structured summary of recent research papers including titles, "
        "main contributions, datasets used, and publication year."
    ),
    agent=literature_miner
)

task_2 = Task(
    description=(
        "Compare methodologies across the collected papers, focusing on "
        "architectures, training strategies, and evaluation metrics."
    ),
    expected_output=(
        "A comparative analysis highlighting similarities, differences, "
        "strengths, and weaknesses of the methodologies."
    ),
    agent=methodology_analyzer
)

task_3 = Task(
    description=(
        "Identify emerging trends, frequently used techniques, and "
        "under-explored directions in the literature."
    ),
    expected_output=(
        "A synthesized list of emerging research trends and under-explored "
        "areas supported by evidence from the literature."
    ),
    agent=trend_synthesizer
)

task_4 = Task(
    description=(
        "Propose clear research gaps and suggest potential experiments "
        "or evaluation setups."
    ),
    expected_output=(
        "A list of clearly articulated research gaps along with suggested "
        "experiments or benchmarks to address them."
    ),
    agent=research_gap_agent
)


In [ ]:
research_crew = Crew(
    agents=[
        literature_miner,
        methodology_analyzer,
        trend_synthesizer,
        research_gap_agent
    ],
    tasks=[
        task_1,
        task_2,
        task_3,
        task_4
    ],
    verbose=True
)


In [ ]:
result = research_crew.kickoff(
    inputs={"topic": "Large Language Models for Code Generation"}
)


 [DEBUG]: == Working Agent: Literature Miner
 [INFO]: == Starting Task: Search for recent (last 3–5 years) research papers on the topic: Large Language Models for Code Generation. Summarize key contributions and datasets used.


> Entering new CrewAgentExecutor chain...
Action: Search the internet
Action Input: {"search_query": "Large Language Models for Code Generation recent research papers"} 


Search results: Title: [2512.00020] Large Language Model for Verilog Code Generation
Link: https://arxiv.org/abs/2512.00020
Snippet: By answering four key research questions, we aim to (1) identify the LLMs used for Verilog generation, (2) examine the datasets and metrics ...
---
Title: Performance and interpretability analysis of code generation large ...
Link: https://www.sciencedirect.com/science/article/pii/S0925231225021332
Snippet: This research provides quantitative and qualitative insights into the capabilities and limitations of open-source code-generating LLMs, informing model ...
-

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.1-8b-instant` in organization `org_01kd7msvpjeyraa4crb3bmka4p` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 6322, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
from IPython.display import Markdown
Markdown(result)
